In [1]:
import tensorflow as tf

/Users/sunchao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/sunchao/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
class TRnnConfig(object):
    '''RNN配置参数'''
    embedding_dim = 64  # 词向量维度
    seq_length = 25
    num_classes = 10
    vocab_size = 5000
    num_layers = 2
    hidden_dim = 128
    rnn = "LSTM"
    dropout_keep_prob = 0.8
    learning_rate = 0.001
    batch_size = 128
    num_epochs = 5
    print_per_batch = 100
    save_per_batch = 10

In [4]:
class TextRnn(object):
    """文本分类模型"""
    def __init__(self, config):
        self.config = config
        
        # 三个待输入的数据源
        self.input_x = tf.placeholder(tf.float32, [None, self.config.seq_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, self.config.num_classes], name="input_y")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        
    def rnn(self):
        """RNN模型"""
        def lstm_cell():
            return tf.nn.rnn_cell.BasicLSTMCell(self.config.hidden_dim, state_is_tuple=True)
        def gru_cell():
            return tf.nn.rnn_cell.GRUCell(self.config.hidden_dim)
        
        def dropout():
            if(self.config.rnn == "GRU"):
                cell = gru_cell()
            else:
                cell = lstm_cell()
            return tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=self.config.dropout_keep_prob)
        
        # 词向量映射
        with tf.device("/cpu:0"):
            embedding = tf.get_variable('embedding', [self.config.vocab_size, self.config.embedding_dim])
            embedding_inputs = tf.nn.embedding_lookup(embedding, self.input_x)
        with tf.name_scope("rnn"):
            # 多层LSTM层
            cells = [dropout() for _ in range(self.config.num_layers)]
            rnn_cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)
            _outputs, _ = tf.nn.dynamic_rnn(cell=rnn_cell, inputs=embedding_inputs, dtype=tf.float32)
            last = _outputs[:,-1,:]  # 取最后一个时序作为输出
            
        with tf.name_scope("score"):
            # 全连接层
            fc = tf.layers.dense(last, self.config.hidden_dim, name="fc1")
            fc = tf.contrib.layers.dropout(fc, keep_prob=self.keep_prob)
            fc = tf.nn.relu(fc)
            
            # 分类器
            self.logits = tf.layers.dense(fc, self.config.num_classes, name='fc2')
            self.y_pred_cls = tf.argmax(tf.nn.softmax(self.logits),1)  #  预测类别
        
        with tf.name_scope("optimize"):
            # 损失函数 交叉熵
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.input_y)
            self.loss = tf.reduce_mean(cross_entropy)
            # 优化器
            self.optimizer = tf.train.AdamOptimizer(self.config.learning_rate).minimize(self.loss)
            
        with tf.name_scope("accuracy"):
            correct_pred = tf.equal(tf.argmax(self.input_y, 1), self.y_pred_cls)
            self.acc = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))
            